In [1]:
# this is the library that we will use to create break times in order to mimic human behaviour 
import time

# this library will be used for getting your credentials but keeping them hidden in the script 
from getpass import getpass

# these are the Classes we will use for interaction with a webpage
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager


# library for interacting with the operating system
import os

# you know pandas it's your best buddy 
import pandas as pd

In [2]:
# open the webdriver
url = 'https://twitter.com/login'
driver = webdriver.Chrome('chromedriver')
driver.get(url)

### Login

In [3]:
username = input('Enter your username: ')
username_box = driver.find_element(By.TAG_NAME, "input")

#clear the search box if it has already something in there
username_box.clear()

# with the method send_keys() you can send the information from the script to the webpage
username_box.send_keys(username)
time.sleep(2)

username_box.send_keys(Keys.ENTER)
# or
#login = driver.find_elements(By.XPATH, "//div[@role='button'][@tabindex='0']")[2]

In [5]:
password = getpass('Enter your password: ')

pass_box = driver.find_elements(By.TAG_NAME, "input")[1]

#clear the paword box if it has already something in there
pass_box.clear()

# here we will send the password to the driver 
pass_box.send_keys(password)
time.sleep(1)

pass_box.send_keys(Keys.ENTER)

### Explore

In [6]:
# explore icon
explore_icon = driver.find_element(By.XPATH, "//a[@href='/explore']")
explore_icon.click()
time.sleep(2)

In [7]:
# search query
search = input('Enter a username: ')

search_bar = driver.find_element(By.XPATH, "//input[@aria-label='Search query']")

search_bar.clear()

search_bar.send_keys(search)
time.sleep(3)

# click on the first profile
user_profile = driver.find_element(By.XPATH, "//div[@data-testid='TypeaheadUser']")
user_profile.click()
time.sleep(2)

### Extract the tweets

In [12]:
# scroll down 
driver.execute_script("window.scrollBy(0,500)","")

In [9]:
# function
def scrap_tweets(count):
    tweets_li = []
    for i in range(count):
        try:
            # driver.execute_script("window.scrollBy(0,600)","")
            # time.sleep(1)
            
            tweets = driver.find_elements(By.XPATH, "//div[@data-testid='tweetText']")[i]
            tweets.click()

            time.sleep(2)

            user = driver.find_element(By.XPATH, "//div[@dir='ltr']")
            username = user.find_element(By.TAG_NAME, "span")
            usernametxt = username.text

            tweets = driver.find_element(By.XPATH, "//div[@data-testid='tweetText']")
            tweet_text = tweets.find_elements(By.TAG_NAME, "span")
            alist = []
            for tw in tweet_text:
                alist.append(tw.text)

            twtxt = ' '.join(alist)

            anothalist = []
            tweet_link = tweets.find_elements(By.TAG_NAME, 'a')
            for tw in tweet_link:
                anothalist.append(tw.text)

            twlink = ', '.join(anothalist)
        

            tweets_li.append({'username': usernametxt, 'text': twtxt, 'link': twlink})
            time.sleep(2)

            back = driver.find_element(By.XPATH, "//div[@aria-label='Back'][@role='button']")
            back.click()

            time.sleep(2)
        except IndexError:
            driver.execute_script("window.scrollBy(0,1000)","")
            time.sleep(2)

    
    return tweets_li
    

In [13]:
# ask how many tweets the user wants to scrap
inp = int(input('How many tweets do you want to scrap?: '))
tweets = scrap_tweets(inp)
print('done!')

done!


In [14]:
tweets_df = pd.DataFrame(tweets)
tweets_df

,username,text,link
0,@carlosguardiola,Genial haber compartido en Business Connection...,@negocios_tv
1,@ironhack,Consider this your #invitation to #Ironhac...,"#invitation, #IronhackDataWeek, #April, #data,..."
2,@ironhack,Time to demystify #APIs .\nIf you’ve ever b...,"#APIs, #holiday, #weather, #webdevelopment, ht..."


In [15]:
driver.close()